<a href="https://colab.research.google.com/github/Musaveer39/PySpark/blob/main/Convert_Sql_Data_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()  # Upload mysqlsampledatabase.sql


Saving mysqlsampledatabase.sql to mysqlsampledatabase.sql


In [2]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
!tar xf spark-3.3.2-bin-hadoop3.tgz
!pip install -q findspark

import os, findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
findspark.init()


In [3]:
import os
os.listdir()  # check if file is here


['.config',
 'spark-3.3.2-bin-hadoop3',
 'spark-3.3.2-bin-hadoop3.tgz',
 '.ipynb_checkpoints',
 'mysqlsampledatabase.sql',
 'sample_data']

In [5]:
# Clean MySQL-specific syntax for SQLite compatibility
import re

# Read the SQL content from the uploaded file
with open('/content/mysqlsampledatabase.sql', 'r', encoding='utf-8') as file:
    sql_script = file.read()

sql_script = sql_script.replace("CREATE DATABASE  IF NOT EXISTS classicmodels;", "")
sql_script = sql_script.replace("USE classicmodels;", "")

# Replace MySQL data types with SQLite-friendly ones
sql_script = re.sub(r"mediumtext", "TEXT", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"mediumblob", "BLOB", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"smallint\(\d+\)", "INTEGER", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"decimal\(\d+,\d+\)", "REAL", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"int\(\d+\)", "INTEGER", sql_script, flags=re.IGNORECASE);

# Remove AUTO_INCREMENT and ENGINE definitions
sql_script = re.sub(r"AUTO_INCREMENT", "", sql_script, flags=re.IGNORECASE);
sql_script = re.sub(r"ENGINE\s*=\s*\w+\s*DEFAULT CHARSET=\w+", "", sql_script, flags=re.IGNORECASE);

# Fix escaped single quotes
sql_script = sql_script.replace("\\'", "''")

In [7]:
import sqlite3
conn = sqlite3.connect("sample.db")
conn.executescript(sql_script)

In [8]:
import os
if os.path.exists("classicmodels.db"):
    os.remove("classicmodels.db")

import sqlite3

conn = sqlite3.connect("classicmodels.db")
conn.executescript(sql_script)
conn.commit()


In [9]:
import pandas as pd

# Get table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()]

# Export each table to CSV
for table_name in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    df.to_csv(f"{table_name}.csv", index=False)
    print(f"{table_name}.csv exported")


productlines.csv exported
products.csv exported
offices.csv exported
employees.csv exported
customers.csv exported
payments.csv exported
orders.csv exported
orderdetails.csv exported


In [10]:
from google.colab import files

for table_name in tables:
    files.download(f"{table_name}.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls

In [11]:
import pandas as pd
from google.colab import files

# Get table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()]

# Export each table to CSV and JSON
for table_name in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Save as CSV
    csv_file = f"{table_name}.csv"
    df.to_csv(csv_file, index=False)

    # Save as JSON (records format is easier for APIs)
    json_file = f"{table_name}.json"
    df.to_json(json_file, orient="records", indent=2)

    print(f"{table_name}: CSV and JSON exported")

# Download all files
for table_name in tables:
    files.download(f"{table_name}.csv")
    files.download(f"{table_name}.json")


productlines: CSV and JSON exported
products: CSV and JSON exported
offices: CSV and JSON exported
employees: CSV and JSON exported
customers: CSV and JSON exported
payments: CSV and JSON exported
orders: CSV and JSON exported
orderdetails: CSV and JSON exported


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import pandas as pd
import zipfile
from google.colab import files

# Get table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()]

# Export CSV and JSON
for table_name in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

    # Save CSV
    df.to_csv(f"{table_name}.csv", index=False)

    # Save JSON
    df.to_json(f"{table_name}.json", orient="records", indent=2)

# Create ZIP
zip_filename = "classicmodels_data.zip"
with zipfile.ZipFile(zip_filename, 'w') as zf:
    for table_name in tables:
        zf.write(f"{table_name}.csv")
        zf.write(f"{table_name}.json")

# Download ZIP
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>